In [0]:
import shutil
import os

# Каталог с набором данных
data_dir = 'train'
# Каталог с данными для обучения
train_dir = 'DPD/images/train'
# Каталог с данными для проверки
val_dir = 'DPD/images/validation'
# Каталог с данными для тестирования
test_dir = 'DPD/images/test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 12500

In [0]:
# Функция для создания 3-х папок (data_dir, val_dir, test_dir) с 2-мя вложенными папками (cat, dog)
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))

create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [0]:
#  Функция копирования данных в указанном диапазоне индексов из исходной 
#  разархивированной папки (trani.zip -> train) в указанные папки, 
#  для размещения картинок
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"),
                    os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"),
                   os.path.join(dest_dir, "dogs"))
# Распеределние картинок: тестовый набор - 15%, проверочный набор - 15% 

start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx) #  Вывод индексов изображений
print(start_test_data_idx) #  Вывод индексов изображений

8750
10625


In [0]:
#  Копирование изображений
copy_images(0, start_val_data_idx, data_dir, train_dir) # 0 - 8750
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir) # 8750 - 10625
copy_images(start_test_data_idx, nb_images, data_dir, test_dir) # 10625 - 12500 

Распознавание собак и кошек на изображениях с помощью признаков, извлеченных предварительно обученной нейронной сетью VGG16

In [0]:
import numpy
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.python.keras.optimizers import Adam

In [0]:
# Размеры изображения
img_width, img_height = 150, 150 # backend Tensorflow, channels Last
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 10
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

Решение 1.
Используя предварительнообученную сеть VGG16, мы решим нашу задачу бинарной классификации (Кошки/Собаки)

In [0]:
vgg16 = VGG16(weights='imagenet', include_top=False,
                  input_shape=(img_width, img_height, 3))
vgg16.trainable = False #  Сеть умеет выделять признаки, 
# поэтому ей не нужно тренироваться для этого опять

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 6s 0us/step


In [0]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
#  Создаем составную нейронную сеть на основе VGG16.
model = Sequential()
#  Добавляем последовательную сеть VGG16 вместо слоя.
model.add(vgg16)
#  Добавляем новые слои, которые будут использоваться для классификации.
#  Первый слой - выпрямление. Нужен для получения плоского вектора,
#  т.е. преобразования из двумерного массива в одномерный
model.add(Flatten())
# Полносвязный слой с 256 нейронами
model.add(Dense(256))
# Функция активации Релу
model.add(Activation('relu'))
# Dropout для снижения переобучения
model.add(Dropout(0.5))
# Выходной слой с 1-им нейроном
model.add(Dense(1))
# Функция - сигмоид, для решения бинарной классификации
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0

In [0]:
#  Этап 3
#  Компилируем нашу нейронную сеть.
#  Для этого выберем невысокую скорость обучения в оптимизаторе Adam.
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Этап 4
#  Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir, target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='binary')

val_generator = datagen.flow_from_directory(
    val_dir, target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir, target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='binary')

Found 17500 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.


In [0]:
#  Этап 5
#  Обучаем составную часть
model.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10, validation_data=val_generator,
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
1750/1750 [==============================] - 181s 103ms/step - loss: 0.4091 - acc: 0.8063 - val_loss: 0.2847 - val_acc: 0.8907
Epoch 2/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.2810 - acc: 0.8813 - val_loss: 0.2488 - val_acc: 0.9011
Epoch 3/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.2473 - acc: 0.8974 - val_loss: 0.2335 - val_acc: 0.9080
Epoch 4/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.2284 - acc: 0.9051 - val_loss: 0.2256 - val_acc: 0.9099
Epoch 5/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.2137 - acc: 0.9137 - val_loss: 0.2209 - val_acc: 0.9117
Epoch 6/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.2014 - acc: 0.9182 - val_loss: 0.2169 - val_acc: 0.9123
Epoch 7/10
1750/1750 [==============================] - 175s 100ms/step - loss: 0.1930 - acc: 0.9206 - val_loss: 0.2129 - val_acc: 0.9157
Epoch 8/10
1750/1750 [============

In [0]:
#  Этап 6
#  Сравнение с тестовым набором
score = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Проверка на тестовых данных score(%): {}".format(score[1]*100))

Проверка на тестовых данных score(%): 91.33333563804626


Предсказание модели

In [0]:
img = image.load_img('DPD/dog.jpg', target_size=(150, 150))
x = image.img_to_array(img)
x = numpy.expand_dims(x, axis=0)
x = preprocess_input(x)
predict = model.predict(x)

print("Результаты распознавания: ", predict) # Собака = 1

Результаты распознавания:  [[1.]]


In [0]:
img = image.load_img('DPD/cat.jpg', target_size=(150, 150))
x = image.img_to_array(img)
x = numpy.expand_dims(x, axis=0)
x = preprocess_input(x)
predict = model.predict(x)

print("Результаты распознавания: ", predict) # Кошка = 0

Результаты распознавания:  [[0.]]


In [0]:
model.save('transferlearning.h5')